**O código a seguir cria e treina um classificador de votação no Scikit-Learn composto
por três classificadores diversos**

In [248]:
import pandas as pd
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [249]:
#df = datasets.make_moons()

**Dividir o dataset em treino e teste**

In [250]:
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size = 0.3, random_state=2)

In [251]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

In [252]:
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), 
                ('rf', rnd_clf), 
                ('svc', svm_clf)], voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

Vejamos a acurácia de cada classificador no conjunto de testes:

**Prevendo a classe com a maior probabilidade na média sobre todos os classificadores individuais**

In [263]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

In [269]:
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', _svm_clf)], voting='soft')
        
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [270]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9777777777777777
RandomForestClassifier 0.9777777777777777
SVC 0.9777777777777777
VotingClassifier 0.9777777777777777


**O código a seguir
treina um conjunto de 500 classificadores de Árvores de Decisão,5 cada um treinado
em 100 instâncias de treinamento amostradas aleatoriamente com substituição no
conjunto de treinamento (exemplo de bagging)**

In [182]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [186]:
#bootstrap=false para um conjunto de trinamento no modo  pasting
# n_jobs=-1, informa a quantidade de núcleos que serão utilizados para as previsões
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

**Avaliação do ensemble com a média das avaliações Out-of-bag `oob_score=True`**

*Avaliação automática após o treinamento*

In [190]:
bag_clf=BaggingClassifier(
DecisionTreeClassifier(), n_estimators=500, 
bootstrap=True, n_jobs=-1, oob_score=True)

In [191]:
# O BaggingClassifier obteve a acurácia de 0.95 no conjunto de treino
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9523809523809523

In [192]:
# O BaggingClassifier obteve a acurácia de 0.95 no conjunto de teste
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9555555555555556

## Florestas Aleatórias

*O código
a seguir treina um classificador de Floresta Aleatória com 500 árvores (cada uma limitada
a 16 nós no máximo) com a utilização de todos os núcleos de CPU disponíveis:*

In [193]:
from sklearn.ensemble import RandomForestClassifier

In [194]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

O BaggingClassifier a seguir é
equivalente ao RandomForestClassifier anterior:

In [195]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

## Importância da Característica

*O código a seguir treina um RandomForestClassifier no conjunto de
dados da íris e exibe cada importância desta característica.*

In [208]:
from sklearn.datasets import load_iris

In [206]:
iris = load_iris()

In [215]:
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_): 
    print(name, score)

sepal length (cm) 0.08622398604531423
sepal width (cm) 0.02264141257373865
petal length (cm) 0.4449600970379279
petal width (cm) 0.4461745043430193


## Boosting e AdaBoost

In [216]:
from sklearn.ensemble import AdaBoostClassifier

In [219]:
ada_clf = AdaBoostClassifier(
DecisionTreeClassifier(max_depth=1), n_estimators=200,
algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

## Gradiente Boosting

In [220]:
from sklearn.tree import DecisionTreeRegressor

In [221]:
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

Treinando um segundo `DecisionTreeRegressor` nos erros residuais cometidos pelo
primeiro previsor:

In [222]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

Treinando o terceiro regressor nos erros residuais cometidos pelo segundo previsor

In [223]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

Agora, temos um ensemble contendo três árvores. Ele pode fazer previsões em uma nova
instância adicionando as previsões de todas as árvores:

In [231]:
y_pred = sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))
y_pred

array([ 0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297, -0.15362812,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00776297,
        0.00776297,  0.00776297,  0.00776297,  0.00776297,  0.00

**Criando o mesmo ensemble anterior, porém com a classse `GradienteBoostingRegressor`**

In [233]:
from sklearn.ensemble import GradientBoostingRegressor

In [235]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

O código a seguir
treina um ensemble GBRT com 120 árvores e mede o erro de validação em cada estágio
do treinamento para encontrar o número ideal de árvores e, utilizando o número ideal
de árvores, finalmente treina outro ensemble GBRT:

In [236]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [237]:
X_train, X_val, y_train, y_val = train_test_split(X,y)

In [238]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [241]:
errors = [mean_squared_error(y_val, y_pred) 
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

In [243]:
gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=109)

**Implementando uma parada antecipada com o parâmetro `warm_start=True`**

Para que serve? 

*Uma parada antecipada interrompe antecipadamente
o treinamento (em vez de treinar primeiro um grande número de árvores e em seguida
olhar para trás para encontrar o número ideal).*

O código a seguir
deixa de treinar quando o erro de validação não melhora por cinco iterações seguidas:

In [244]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

In [246]:
min_val_error = float("inf")
error_going_up = 0
for n_stmators in range(1, 120):
    gbrt.n_stmators = n_stmators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error: 
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up +=1
        if error_going_up == 5:
            break #parada antecipada